In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os
from tqdm import tqdm
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def generate(name):
    eventId_to_bert = torch.load("../bert_raw_data.pth")
    num_sessions = 0
    inputs = []
    outputs = []
    with open(name, 'r') as f:
        for line in tqdm(f,desc="load data"):
            num_sessions += 1
            line = tuple(map(lambda n: n, map(int, line.strip().split())))
            for i in range(len(line) - window_size):
                event_ids = line[i:i + window_size]
                bert_input = []
                for id in event_ids:
                    bert_input.append(eventId_to_bert['E'+str(id)][1][0].cpu().numpy())
                inputs.append(tuple(bert_input))
                outputs.append(line[i + window_size])
    print('Number of sessions({}): {}'.format(name, num_sessions))
    print('Number of seqs({}): {}'.format(name, len(inputs)))
    dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
    return dataset


class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [3]:
# Hyperparameters
num_classes = 28
num_epochs = 50
batch_size = 1024
input_size = 768
model_dir = 'model'
log = 'Bert_Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
num_layers = 2
hidden_size = 64
window_size = 10
file_dir = 'data/'

In [25]:
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
model.load_state_dict(torch.load(model_dir + '/' + 'bert_total' + '.pt'))
seq_dataset = generate(file_dir+'hdfs_train')
dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

load data: 4855it [03:29, 23.20it/s]


Number of sessions(data/hdfs_train): 4855
Number of seqs(data/hdfs_train): 46575


In [27]:
writer = SummaryWriter(log_dir='log/' + log)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# Train the model
total_step = len(dataloader)

In [35]:
start_time = time.time()
current_epoch = 50
for epoch in range(current_epoch,current_epoch+num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        writer.add_graph(model, seq)
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
    writer.add_scalar('train_loss', train_loss / total_step, epoch + 1)
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12288000 bytes. Buy new RAM!


In [29]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + 'bert_total' + '.pt')
writer.close()
print('Finished Training')

Finished Training


In [10]:
import gc

In [11]:
gc.collect()

22

In [4]:
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
model.load_state_dict(torch.load(model_dir + '/' + 'bert_total' + '.pt'))

<All keys matched successfully>

In [5]:
def generate_test_data(name):
    hdfs = set()
    # hdfs = []
    with open('data/' + name, 'r') as f:
        for ln in f.readlines():
            ln = list(map(lambda n: n, map(int, ln.strip().split())))
            ln = ln + [-1] * (window_size + 1 - len(ln))
            hdfs.add(tuple(ln))
            # hdfs.append(tuple(ln))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [7]:
model.eval()
# print('model_path: {}'.format(model_path))
test_normal_loader = generate_test_data('hdfs_test_normal')
# test_abnormal_loader = generate_test_data('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 14177


In [8]:
eventId_to_bert = torch.load("../bert_raw_data.pth")

In [9]:
print(eventId_to_bert.keys())

dict_keys(['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30'])


In [15]:
# model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
# print('model_path: {}'.format(model_path))
test_normal_loader = generate_test_data('hdfs_test_normal')
test_abnormal_loader = generate_test_data('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 14177
Number of sessions(hdfs_test_abnormal): 4123


In [19]:
# model.load_state_dict(torch.load('E:\\大四上\\毕设\\代码\\DeepLog-master\\bert\\model\\Small_Adam_batch_size=400_epoch=300.pt'))
model.load_state_dict(torch.load(model_dir + '/' + log + '.pt'))
model.eval()

Model(
  (lstm): LSTM(768, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=28, bias=True)
)

In [10]:
model.eval()

Model(
  (lstm): LSTM(768, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=28, bias=True)
)

In [11]:
padding = torch.zeros_like(eventId_to_bert['E5'][1][0].cpu()).numpy()

In [25]:
pattern = torch.FloatTensor(list(test_normal_loader)[2])
pattern = pattern.view(1,-1)

In [26]:
pattern = list(pattern.numpy()[0])

In [28]:
for window_size in range(1,len(pattern)):
    test_data = pattern[:window_size]
    eventIds = test_data
    seq = []
    for id in eventIds:
        if id==-1:
            seq.append(padding)
        else:
            seq.append(eventId_to_bert['E'+str(int(id))][1][0].cpu().numpy())
    seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
    output = model(seq).cpu()
    predicted = torch.argsort(output, 1)[:,-5:]
    print('window size:'+ str(window_size))
    print('seq: '+str(test_data),end=' ')
    print('expected num: '+str(pattern[window_size]))
    print('prediction:' + str(predicted.numpy()))
    if pattern[window_size] in predicted[0]:
        print('correct:'+str(5-list(predicted[0].numpy()).index(pattern[window_size])))
    else:
        print('wrong')
    print()

window size:1
seq: [5.0] expected num: 22.0
prediction:[[ 4 16 26 23 21]]
wrong

window size:2
seq: [5.0, 22.0] expected num: 5.0
prediction:[[18 26  4  9 21]]
wrong

window size:3
seq: [5.0, 22.0, 5.0] expected num: 5.0
prediction:[[ 4 16 11 26 21]]
wrong

window size:4
seq: [5.0, 22.0, 5.0, 5.0] expected num: 11.0
prediction:[[ 9 16 11 26 21]]
correct:3

window size:5
seq: [5.0, 22.0, 5.0, 5.0, 11.0] expected num: 9.0
prediction:[[ 4 11  9 26 21]]
correct:3

window size:6
seq: [5.0, 22.0, 5.0, 5.0, 11.0, 9.0] expected num: 11.0
prediction:[[ 3  9 11 26 21]]
correct:3

window size:7
seq: [5.0, 22.0, 5.0, 5.0, 11.0, 9.0, 11.0] expected num: 11.0
prediction:[[25 21 11 26  9]]
correct:3

window size:8
seq: [5.0, 22.0, 5.0, 5.0, 11.0, 9.0, 11.0, 11.0] expected num: 9.0
prediction:[[25 21 11 26  9]]
correct:1

window size:9
seq: [5.0, 22.0, 5.0, 5.0, 11.0, 9.0, 11.0, 11.0, 9.0] expected num: 9.0
prediction:[[ 4  3  9 11 26]]
correct:3

window size:10
seq: [5.0, 22.0, 5.0, 5.0, 11.0, 9.0, 1

In [11]:
num_candidates = 5
TP = 0
FP = 0
FP_log = []
FN_log = []

In [16]:
num_candidates = 5
TP = 0
FP = 0
FP_log = []
FN_log = []
# Test the model
torch.zeros_like(eventId_to_bert['E5'][1][0].cpu())
start_time = time.time()
with torch.no_grad():
    for line in tqdm(test_normal_loader,"normal:"):

        for i in range(len(line) - window_size):
            eventIds = line[i:i + window_size]
            label = line[i + window_size]
            seq = []
            for id in eventIds:
                if id==-1:
                    seq.append(padding)
                else:
                    seq.append(eventId_to_bert['E'+str(id)][1][0].cpu().numpy())

            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                FP += 1
                FP_log.append(line)
                break
exception_num = 0
with torch.no_grad():
    for line in tqdm(test_abnormal_loader,"abnormal:"):
        flag = True
        for i in range(len(line) - window_size):
            eventIds = line[i:i + window_size]
            label = line[i + window_size]
            seq = []
            for id in eventIds:
                if id==-1:
                    seq.append(padding)
                else:
                    seq.append(eventId_to_bert['E'+str(id)][1][0].cpu().numpy())
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                TP += 1
                flag = False
                break
        if flag:
           FN_log.append(line)
elapsed_time = time.time() - start_time
print(exception_num)
print('elapsed_time: {:.3f}s'.format(elapsed_time))
# Compute precision, recall and F1-measure
FN = len(test_abnormal_loader) - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 4123/4123 [10:20<00:00,  6.65it/s]

0
elapsed_time: 3728.669s
false positive (FP): 1033, false negative (FN): 22, Precision: 79.879%, Recall: 99.466%, F1-measure: 88.603%
Finished Predicting


In [21]:
((14177+4123)-(1033+22))/(14177+4123)

0.9423497267759563

In [12]:
with torch.no_grad():
    for line in tqdm(test_abnormal_loader,"abnormal:"):
        flag = True
        for i in range(len(line) - window_size):
            eventIds = line[i:i + window_size]
            label = line[i + window_size]
            seq = []
            for id in eventIds:
                if id==-1:
                    seq.append(padding)
                else:
                    seq.append(eventId_to_bert['E'+str(id)][1][0].cpu().numpy())
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                TP += 1
                flag = False
                break
        if flag:
           FN_log.append(line)

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 4123/4123 [09:15<00:00,  7.43it/s]


In [18]:
FP

645

In [13]:
TP

4101

In [21]:
with open("FN.txt",'w') as f:
    for line in FN_log:
        line = list(map(str, line))
        f.write(" ".join(line)+'\n')
with open("FP.txt",'w') as f:
    for line in FP_log:
        line = list(map(str, line))
        f.write(" ".join(line)+'\n')